# 내가 짠 뉴스 유사도 비교 코드

최종본은 main(최종).ipynb 입니다.


### 최종본과 다른점

TfidfVectorizer 를 활용하고 안하고의 차이!

또한 100% 활용한 최종본이 결과가 좀 더 좋게 나왔다.

이유는 내가 구현한 코드는 정규화 작업이 없는데 모듈에는 있다고한다.(L2)

그 차이가 결과를 만든것 같다.

In [1]:
import pandas as pd
import numpy as np
from math import log
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('./pretreatment_data/yeonhap_pre.csv', names=['CNo', 'Subject', 'Contents'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64122 entries, 0 to 64121
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CNo       64122 non-null  object
 1   Subject   64122 non-null  object
 2   Contents  64122 non-null  object
dtypes: object(3)
memory usage: 1.5+ MB


In [3]:
et_df = pd.read_csv('./pretreatment_data/etoday_pre.csv', names=['CNo', 'Subject', 'Contents'])
et_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9747 entries, 0 to 9746
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CNo       9747 non-null   object
 1   Subject   9747 non-null   object
 2   Contents  9747 non-null   object
dtypes: object(3)
memory usage: 228.6+ KB


### 연합 뉴스 idf 값을 저장하기 위한 자체 코드

이투데이 뉴스의 tf-idf 를 구하려면 연합 뉴스로 만든 idf 값이 필요하다. 그래서 만듦.

하지만 TfidfVectorizer 에 있는 기능이었다.

정확히는 연합 뉴스로 학습한 값으로 이투데이 뉴스를 (idf)계산가능 했던것~

이젠 알았음


In [4]:
def tf(term, document):
    # document 에서 term 의 등장 횟수를 count
    return document.count(term)

def idf(term, documents):
    # doc_list 에서 term 이 등장한 문서 수를 count
    
    # 함수가 돌 때마다 초기화
    dfn = 0
    for doc in documents:
        # 각 문서마다 해당 언어가 있는지 확인
        if term in doc:
            dfn = dfn + 1
    return log(len(documents)/(dfn+1))
    
def tfidf(term, docu):
    # tf * idf
    return tf(term, docu)*idf(term)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

tfidf_matrix = tfidf.fit_transform(df['Contents'])
print(tfidf_matrix.shape)

(64122, 65825)


token_list_all 에는 연합 뉴스로 구성된 단어들이 존재한다.

단어 집합이라 중복 없고, 딕셔너리 형태로 (단어, 순번) 이 저장되어있음

In [6]:
# 토큰(단어) 를 아이템 오름차순으로 키를 정렬 토큰(단어) 를 아이템 오름차순으로 키를 정렬
token_list_all = sorted(tfidf.vocabulary_.items(), key = lambda item: item[1])
len(token_list_all)

65825

In [ ]:
for token, _ in token_list_all:
    print(token)
    break

idf_dit_all 은 미리 계산한 연합뉴스의 idf matrix임.

딕셔너리 형태로 저장했고, (단어, idf 값) 이런 형식이다.

In [7]:
with open("idf_dit_all.pickle", 'rb') as handle:
    dit_all = pickle.load(handle) 

def aaa(num):
    vs_list = []

    doc = et_df.iloc[num]['Contents']
    vs_list.append([])
    for token, _ in token_list_all:
        vs_list[-1].append( tf(token, doc)*dit_all[token] )

    cosine_matrix = cosine_similarity(vs_list, tfidf_matrix)

    # news title과 id를 맵핑할 dictionary를 생성
    news2id = {}
    for i, c in enumerate(df['CNo']):
        news2id[i] = c

    # id와 news title를 매핑할 dictionary를 생성
    id2news = {}
    for i, c in news2id.items():
        id2news[c] = i

    sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[0])]
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)

    sim_scores = [(news2id[i], score) for i, score in sim_scores[0:5]]

    return sim_scores

In [10]:
for num in range(9747):
    sim_scores = aaa(num)
    sim = []
    if sim_scores[0][1] > 0.9:
        sim.append(sim_scores[0][0])
        result = pd.DataFrame({
            'CNo' : et_df.iloc[num]['CNo'],
            'Subject' : et_df.iloc[num]['Subject'],
            'similar' : sim
        })
        # 바로 저장
        result.to_csv("./similar_result_90.csv", encoding='utf-8',mode = 'a', index=False, header=False)

# 끝